In [1]:
import sys, os
sys.path.append(os.pardir)

In [2]:
import numpy as np
from common.layers import *
from common.gradient import numerical_gradient
from collections import OrderedDict

In [5]:
class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01):
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)

        self.layers = OrderedDict()
        self.layers['A1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['R1'] = Relu()
        self.layers['A2'] = Affine(self.params['W2'], self.params['b2'])
        self.lastLayer = SoftmaxWithLoss()
        
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        return x
    
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1 : t = np.argmax(t, axis=1)
            
        accuracy = np.sum(y==t) / float(x.shape[0])
        return accuracy
    
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads
    
    def gradient(self, x, t):
        self.loss(x, t)
        
        dout = 1
        dout = self.lastLayer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)
            
        
        grads = {}
        grads['W1'] = self.layers['A1'].dW
        grads['b1'] = self.layers['A1'].db
        grads['W2'] = self.layers['A2'].dW
        grads['b2'] = self.layers['A2'].db
        
        return grads

In [7]:
from dataset.mnist import load_mnist

(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size/batch_size, 1)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    grad = network.gradient(x_batch, t_batch)
    
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
        
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print("Train Acc, Test Acc | " + str(train_acc) + ", " + str(test_acc))

Train Acc, Test Acc | 0.13928333333333334, 0.1369
Train Acc, Test Acc | 0.9045666666666666, 0.907
Train Acc, Test Acc | 0.9210833333333334, 0.921
Train Acc, Test Acc | 0.9349833333333334, 0.9315
Train Acc, Test Acc | 0.9400333333333334, 0.9397
Train Acc, Test Acc | 0.9488, 0.9472
Train Acc, Test Acc | 0.95555, 0.9531
Train Acc, Test Acc | 0.9588666666666666, 0.9561
Train Acc, Test Acc | 0.9635666666666667, 0.9584
Train Acc, Test Acc | 0.9654166666666667, 0.9607
Train Acc, Test Acc | 0.9677666666666667, 0.9632
Train Acc, Test Acc | 0.9691333333333333, 0.9645
Train Acc, Test Acc | 0.9720833333333333, 0.9665
Train Acc, Test Acc | 0.9735833333333334, 0.9667
Train Acc, Test Acc | 0.9743, 0.9674
Train Acc, Test Acc | 0.9761333333333333, 0.9687
Train Acc, Test Acc | 0.9779, 0.9693
